Import libraries

In [1]:
import numpy as np
import math
import sympy as sp
from IPython.display import Latex, display_latex

np.set_printoptions(suppress=True, linewidth=np.nan)


# Dantzig's simplex algorithm

### Utilities

In [2]:
def build_tableau(A: np.ndarray, b: np.ndarray, c: np.ndarray) -> np.ndarray:
    tableau = np.r_[[c], A]
    tableau = np.c_[tableau, np.insert(b, 0, 0)]
    return tableau


In [3]:
def minimum_ratio_test(col: np.ndarray, b: np.ndarray) -> int:
    r_min = 0
    min_val = math.inf
    for k in range(len(b)):
        if col[k] > 0:
            min_val_temp = b[k] / col[k]
            if min_val_temp < min_val:
                min_val = min_val_temp
                r_min = k
    return r_min if min_val != math.inf else None


In [4]:
def pivoting(tableau: np.ndarray, row: int, col: int) -> np.ndarray:
    # escale pivot row min to 1.0
    tableau[row] = tableau[row] / tableau[row, col]
    # pivot proccess: convert al column to zero except row
    for k in range(len(tableau)):
        if k != row and abs(tableau[k, col]) > 0:
            tableau[k] = tableau[k] - tableau[k, col] * tableau[row, :]
    return tableau


In [5]:
def correct_cost(tableau: np.ndarray, basic_var: list[int]):
    # correct basic variable cost with value distict of zero 
    for index, col in enumerate(basic_var):
        if abs(tableau[0, col]) > 0:
            row = index + 1
            tableau[0] = tableau[0] - tableau[0, col] * tableau[row, :]
    print(f"Cost corrected")
    return tableau


In [6]:
def correct_all_tableau(tableau: np.ndarray, basic_var: list[int]):
    # correct basic variable of all column with value distict of zero
    for index, col in enumerate(basic_var):
        row = index + 1
        tableau = pivoting(tableau, row, col)
    print("All tableau corrected")
    return tableau


In [7]:
def basic2vertex(tableau: np.ndarray, basic_var: list[int]) -> None:
    # only work with simplex method because m = len(basic_var) and n = len(non_basic_var)
    dim_1, dim_2 = tableau.shape
    m = len(basic_var)
    n = dim_2 - dim_1  # for Big-M don't delete artificial variables
    x = np.zeros(m + n)
    x[basic_var] = tableau[1:, -1]
    return x[:n]


In [8]:
def optimal_solution(tableau: np.ndarray, basic_var: list[int]) -> None:
    print("Optimal solution: x_opt =", basic2vertex(tableau, basic_var))


In [9]:
def real_index(basic_var: list[int]):
    return list(map(lambda x: x + 1, basic_var))


For unbounded model

In [10]:
def extreme_direction(tableau: np.ndarray, basic_var: list[int], col: int) -> None:
    # only work with simplex method because m = len(basic_var) and n = len(non_basic_var)
    dim_1, dim_2 = tableau.shape
    m = len(basic_var)
    n = dim_2 - dim_1
    x = np.zeros(m + n)
    dir = np.zeros(m + n)
    x[basic_var] = tableau[1:, -1]
    dir[basic_var] = tableau[1:, col]
    print(f"L = {x} + λ{-dir}\nL = {x[:n]} + λ{-dir[:n]}")


### Simplex Algorithm

In [11]:
def simplex(tableau: np.ndarray, basic_var: list[int]) -> bool:
    print(f"x_B = {real_index(basic_var)}\n{tableau}")
    while (tableau[0, :-1] > 0).any():
        # max criterion
        c_max = np.argmax(tableau[0, :-1])
        # minimum ratio test
        r_min = minimum_ratio_test(col=tableau[1:, c_max], b=tableau[1:, -1])
        if r_min == None:
            print("The model is unbounded")
            extreme_direction(tableau, basic_var, c_max)
            break
        # pivoting
        tableau = pivoting(tableau, r_min + 1, c_max)
        # swap row with col
        basic_var[r_min] = c_max
        print(f"x_B = {real_index(basic_var)}\n{tableau}")
    return tableau


### Big-M
The big-M procedure, each such constraint $i$ is augmented, together with its slack variable, with a so-called artificial variable $u_i$,  and the objective function is augmented with $−Mu_i$, where $M$ is a big positive real number. For big values of $M$ the simplex algorithm will put highest priority on making the value of the factor $Mu_i$ as small as possible, thereby setting the value of $u_i$ equal to zero. Big-M and two-phases are used when $0$ isn't feasible basic solution.

In [12]:
def big_M(tableau: np.ndarray, basic_var: list[int], artificial_var: list[int]) -> np.ndarray:
    print(f"x_B = {real_index(basic_var)}\n{tableau}")
    # correct first row of tableau for each artificial var because the artificial_var position is -M
    tableau = correct_cost(tableau, basic_var)
    # solve with simplex
    simplex(tableau, basic_var)
    # delete the artificial variable
    print("Delete artificial variables", artificial_var)
    tableau = np.delete(tableau, artificial_var, axis=1)
    print(f"x_B = {real_index(basic_var)}\n{tableau}")
    return tableau


### Two-Phases
The two-phase procedure add, in the same manner as employed in the big-M procedure, but instead of adding each artificial variable to the objetive with a large negative coefficient, the objetive function is replace by minus the sum of all artificial variables . During the **first phase**, the simplex algorithm tries to maximize this objetive, effectively trying to give all artificial variables the value zero.  By doing this a feasible basic solution is founded. When this has happened, the **second-phase** starts by replacing the artificial objetive function by the objetive function of the original model and solve this with simplex method.|

In [13]:
def two_phases(tableau: np.ndarray, c: np.ndarray, basic_var: list[int], artificial_var: list[int]):
    print(f"x_B = {real_index(basic_var)}\n{tableau}")
    # correct first row of tableau because the artificial_var position is -1
    tableau = correct_cost(tableau, basic_var)
    print(f"x_B = {real_index(basic_var)}\n{tableau}")

    print("Start phase One")
    # while until each artifical var be negative and not be basic var
    while any(((u in basic_var) or (tableau[0, u] > 0)) for u in artificial_var):
        # max criterion
        c_max = np.argmax(tableau[0, :-1])
        # minimum ratio test
        r_min = minimum_ratio_test(col=tableau[1:, c_max], b=tableau[1:, -1])
        # pivoting
        tableau = pivoting(tableau, r_min + 1, c_max)
        # swap row with col
        basic_var[r_min] = c_max
        print(f"x_B = {real_index(basic_var)}\n{tableau}")

    print("Delete artificial variables", artificial_var)
    # delete the artificial variable because we found a solution factible
    tableau = np.delete(tableau, artificial_var, axis=1)
    print("Start phase two, put original cost")
    # put c in first row of tableau
    tableau[0, :len(c)] = c
    # solve with simplex
    print(f"x_B = {real_index(basic_var)}\n{tableau}")
    tableau = correct_cost(tableau, basic_var)
    return simplex(tableau, basic_var)


### Book: Linear and Integer Optimization


#### Dovetail Model 
$$\begin{align*}
\max  3 x_{1}&+2 x_{2}& \\
\text { s.t. } \quad x_{1}&+x_{2}& \leq 9 \\
3 x_{1}&+x_{2}& \leq 18 \\
x_{1}&{} {}& \leq 7 \\
{}& \quad x_{2} & \leq 6 \\
x_{1}&, x_{2} \geq 0
\end{align*}$$

In [14]:
A = [[1, 1, 1, 0, 0, 0],
     [3, 1, 0, 1, 0, 0],
     [1, 0, 0, 0, 1, 0],
     [0, 1, 0, 0, 0, 1.0]]

b = [9, 18, 7, 6.]

c = [3, 2, 0, 0, 0, 0.]

basic_var = [2, 3, 4, 5]

tableau = build_tableau(A, b, c)
simplex(tableau, basic_var)
optimal_solution(tableau, basic_var)


x_B = [3, 4, 5, 6]
[[ 3.  2.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  6.]]
x_B = [3, 1, 5, 6]
[[  0.           1.           0.          -1.           0.           0.         -18.        ]
 [  0.           0.66666667   1.          -0.33333333   0.           0.           3.        ]
 [  1.           0.33333333   0.           0.33333333   0.           0.           6.        ]
 [  0.          -0.33333333   0.          -0.33333333   1.           0.           1.        ]
 [  0.           1.           0.           0.           0.           1.           6.        ]]
x_B = [2, 1, 5, 6]
[[  0.    0.   -1.5  -0.5   0.    0.  -22.5]
 [  0.    1.    1.5  -0.5   0.    0.    4.5]
 [  1.    0.   -0.5   0.5   0.    0.    4.5]
 [  0.    0.    0.5  -0.5   1.    0.    2.5]
 [  0.    0.   -1.5   0.5   0.    1.    1.5]]
Optimal solution: x_opt = [4.5 4.5]


**Two-Phases**: Dovetail Model modified: Constraint added $x_1 + x_2 \ge 5$

In [15]:
tableau = np.array([[0, 0, 0, 0, 0, 0, 0, -1, 0],
                    [1, 1, 1, 0, 0, 0, 0, 0, 9.],
                    [3, 1, 0, 1, 0, 0, 0, 0, 18],
                    [1, 0, 0, 0, 1, 0, 0, 0, 7.],
                    [0, 1, 0, 0, 0, 1, 0, 0, 6.],
                    [1, 1, 0, 0, 0, 0, -1, 1, 5]])

c = [3, 2, 0, 0, 0, 0, 0.]

basic_var = [2, 3, 4, 5, 7]
artificial_var = [7]

tableau = two_phases(tableau, c, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [3, 4, 5, 6, 8]
[[ 0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  0.  0.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
Cost corrected
x_B = [3, 4, 5, 6, 8]
[[ 1.  1.  0.  0.  0.  0. -1.  0.  5.]
 [ 1.  1.  1.  0.  0.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
Start phase One
x_B = [3, 4, 5, 6, 1]
[[ 0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  1.  0.  0.  0.  1. -1.  4.]
 [ 0. -2.  0.  1.  0.  0.  3. -3.  3.]
 [ 0. -1.  0.  0.  1.  0.  1. -1.  2.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
Delete artificial variables [7]
Start phase two, put original cost
x_B = [3, 4, 5, 6, 1]
[[ 3.  2.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  1.  4.]
 [ 0. -2.  0.  1.  0.  0.  3.  3.]
 [ 0

**BIG-M**:  Ejemplo visto el ciclo pasado Lineal 6 

In [16]:
M = 10
A = [[2, -1, -1, 0, 0, 1, 0],
     [-1, 2, 0, -1, 0, 0, 1],
     [1, 1., 0., 0, 1, 0, 0.]]

b = [4, 2, 12.]

c = [2, 1, 0, 0, 0, -M, -M]

basic_var = [5, 6, 4]
artificial_var = [5, 6]

tableau = build_tableau(A, b, c)
tableau = big_M(tableau, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [6, 7, 5]
[[  2.   1.   0.   0.   0. -10. -10.   0.]
 [  2.  -1.  -1.   0.   0.   1.   0.   4.]
 [ -1.   2.   0.  -1.   0.   0.   1.   2.]
 [  1.   1.   0.   0.   1.   0.   0.  12.]]
Cost corrected
x_B = [6, 7, 5]
[[ 12.  11. -10. -10.   0.   0.   0.  60.]
 [  2.  -1.  -1.   0.   0.   1.   0.   4.]
 [ -1.   2.   0.  -1.   0.   0.   1.   2.]
 [  1.   1.   0.   0.   1.   0.   0.  12.]]
x_B = [1, 7, 5]
[[  0.   17.   -4.  -10.    0.   -6.    0.   36. ]
 [  1.   -0.5  -0.5   0.    0.    0.5   0.    2. ]
 [  0.    1.5  -0.5  -1.    0.    0.5   1.    4. ]
 [  0.    1.5   0.5   0.    1.   -0.5   0.   10. ]]
x_B = [1, 2, 5]
[[  0.           0.           1.66666667   1.33333333   0.         -11.66666667 -11.33333333  -9.33333333]
 [  1.           0.          -0.66666667  -0.33333333   0.           0.66666667   0.33333333   3.33333333]
 [  0.           1.          -0.33333333  -0.66666667   0.           0.33333333   0.66666667   2.66666667]
 [  0.           0.           1.           1.    

#### Equality constraints
Example 3.8.1. Consideer the following LO-model 
\begin{align*}
\max  40 x_{1}& + 100 x_{2}  + 150 x_{3}  \\
\text { s.t. }\quad x_{1} & + 2 x_{2}  +2 x_{3} =3 \\
 30 x_{1}& + 10 x_{2}  + 20 x_{3} =75 \\
 x_{1}&, x_{2}, x_{3} \geq 0
\end{align*}

Method 1: Big-M

Add artificial variable $u_1$ and $u_2$, the simplex algorithm will put highest priority on making the value of the factor $Mu_i$ as small as possible, thereby setting the value of $u_i$ equal to zero to maintain equality.

In [17]:
M = 100
A = [[1.0, 2, 2., 1, 0],
     [30, 10, 20, 0, 1]]

b = [3, 75.]
c = [40, 100, 150., -M, -M]

basic_var = [3, 4]

tableau = build_tableau(A, b, c)
tableau = big_M(tableau, basic_var, [])
optimal_solution(tableau, basic_var)


x_B = [4, 5]
[[  40.  100.  150. -100. -100.    0.]
 [   1.    2.    2.    1.    0.    3.]
 [  30.   10.   20.    0.    1.   75.]]
Cost corrected
x_B = [4, 5]
[[3140. 1300. 2350.    0.    0. 7800.]
 [   1.    2.    2.    1.    0.    3.]
 [  30.   10.   20.    0.    1.   75.]]
x_B = [4, 1]
[[   0.          253.33333333  256.66666667    0.         -104.66666667  -50.        ]
 [   0.            1.66666667    1.33333333    1.           -0.03333333    0.5       ]
 [   1.            0.33333333    0.66666667    0.            0.03333333    2.5       ]]
x_B = [3, 1]
[[   0.     -67.5      0.    -192.5    -98.25  -146.25 ]
 [   0.       1.25     1.       0.75    -0.025    0.375]
 [   1.      -0.5      0.      -0.5      0.05     2.25 ]]
Delete artificial variables []
x_B = [3, 1]
[[   0.     -67.5      0.    -192.5    -98.25  -146.25 ]
 [   0.       1.25     1.       0.75    -0.025    0.375]
 [   1.      -0.5      0.      -0.5      0.05     2.25 ]]
Optimal solution: x_opt = [2.25  0.    0.375]


Method 2: Simplex

In [18]:
A = [[1, 2, 2],
     [30, 10, 20.]]

b = [3, 75.]

c = [40, 100, 150.]

basic_var = [0, 1]

tableau = build_tableau(A, b, c)
print(f"x_B = {basic_var}\n{tableau}")

# correcting the coefficients different from zero in the basic var
tableau = correct_all_tableau(tableau, basic_var)
# simplex proccess
tableau = simplex(tableau, basic_var)

# Optimal solution for equality constraints
x = np.zeros_like(c)
x[basic_var] = tableau[1:, -1]
print("Optimal solution: x_opt =", x)


x_B = [0, 1]
[[ 40. 100. 150.   0.]
 [  1.   2.   2.   3.]
 [ 30.  10.  20.  75.]]
All tableau corrected
x_B = [1, 2]
[[   0.     0.    54.  -126. ]
 [   1.     0.     0.4    2.4]
 [  -0.     1.     0.8    0.3]]
x_B = [1, 3]
[[   0.     -67.5      0.    -146.25 ]
 [   1.      -0.5      0.       2.25 ]
 [  -0.       1.25     1.       0.375]]
Optimal solution: x_opt = [2.25  0.    0.375]


#### Unbounded model

In [19]:
A = [[1, -1, 1, 0.],
     [-2, 1, 0, 1.]]

b = [1, 2.0]

c = [0, 1, 0, 0.0]

tableau = build_tableau(A, b, c)

basic_var=[2, 3]

simplex(tableau, basic_var);



x_B = [3, 4]
[[ 0.  1.  0.  0.  0.]
 [ 1. -1.  1.  0.  1.]
 [-2.  1.  0.  1.  2.]]
x_B = [3, 2]
[[ 2.  0.  0. -1. -2.]
 [-1.  0.  1.  1.  3.]
 [-2.  1.  0.  1.  2.]]
The model is unbounded
L = [0. 2. 3. 0.] + λ[-0.  2.  1. -0.]
L = [0. 2.] + λ[-0.  2.]


### CM4E1 Curso: Programación Lineal y flujo en redes, Práctica Dirigida 4

##### 2. Hallar una solución aplicando las tablas del método simplex

a) Simplex method


In [20]:
A = [[2, 1, 1, 0, 0],
     [1, 0, 0, 1, 0],
     [0, 1, 0, 0, 1.]]

b = [9, 4, 3.]

c = [4, 5, 0, 0, 0.]

basic_var = [2, 3, 4]

tableau = build_tableau(A, b, c)
tableau = simplex(tableau, basic_var)
optimal_solution(tableau, basic_var)


x_B = [3, 4, 5]
[[4. 5. 0. 0. 0. 0.]
 [2. 1. 1. 0. 0. 9.]
 [1. 0. 0. 1. 0. 4.]
 [0. 1. 0. 0. 1. 3.]]
x_B = [3, 4, 2]
[[  4.   0.   0.   0.  -5. -15.]
 [  2.   0.   1.   0.  -1.   6.]
 [  1.   0.   0.   1.   0.   4.]
 [  0.   1.   0.   0.   1.   3.]]
x_B = [1, 4, 2]
[[  0.    0.   -2.    0.   -3.  -27. ]
 [  1.    0.    0.5   0.   -0.5   3. ]
 [  0.    0.   -0.5   1.    0.5   1. ]
 [  0.    1.    0.    0.    1.    3. ]]
Optimal solution: x_opt = [3. 3.]


b) Two-phases

In [21]:
# problema b 
A = [[1., 1., 1, 0, 0, 0],
     [-2, -1, 0, 1, 0, 0],
     [0., 1., 0, 0, -1, 1]]

b = [9, 4, 3.]

c = [4, 5, 0, 0.0, 0.0]
c_artificial = [0., 0., 0, 0, 0, -1]

basic_var = [2, 3, 5]
artificial_var = [5]

tableau = build_tableau(A, b, c_artificial)
tableau = two_phases(tableau, c, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [3, 4, 6]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [-2. -1.  0.  1.  0.  0.  4.]
 [ 0.  1.  0.  0. -1.  1.  3.]]
Cost corrected
x_B = [3, 4, 6]
[[ 0.  1.  0.  0. -1.  0.  3.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [-2. -1.  0.  1.  0.  0.  4.]
 [ 0.  1.  0.  0. -1.  1.  3.]]
Start phase One
x_B = [3, 4, 2]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  0.  1.  0.  1. -1.  6.]
 [-2.  0.  0.  1. -1.  1.  7.]
 [ 0.  1.  0.  0. -1.  1.  3.]]
Delete artificial variables [5]
Start phase two, put original cost
x_B = [3, 4, 2]
[[ 4.  5.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  1.  6.]
 [-2.  0.  0.  1. -1.  7.]
 [ 0.  1.  0.  0. -1.  3.]]
Cost corrected
x_B = [3, 4, 2]
[[  4.   0.   0.   0.   5. -15.]
 [  1.   0.   1.   0.   1.   6.]
 [ -2.   0.   0.   1.  -1.   7.]
 [  0.   1.   0.   0.  -1.   3.]]
x_B = [5, 4, 2]
[[ -1.   0.  -5.   0.   0. -45.]
 [  1.   0.   1.   0.   1.   6.]
 [ -1.   0.   1.   1.   0.  13.]
 [  1.   1.   1.   0.   0.   9.]]
Optimal solution: x_opt = [0. 9.]


b) With Big-M method

In [22]:
M = 10

A = [[1., 1., 1, 0, 0, 0],
     [-2, -1, 0, 1, 0, 0],
     [0., 1., 0, 0, -1, 1]]

b = [9, 4, 3.]

c = [4, 5, 0, 0, 0, -M]

basic_var = [2, 3, 5]
artificial_var = [5]

tableau = build_tableau(A, b, c)
tableau = big_M(tableau, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [3, 4, 6]
[[  4.   5.   0.   0.   0. -10.   0.]
 [  1.   1.   1.   0.   0.   0.   9.]
 [ -2.  -1.   0.   1.   0.   0.   4.]
 [  0.   1.   0.   0.  -1.   1.   3.]]
Cost corrected
x_B = [3, 4, 6]
[[  4.  15.   0.   0. -10.   0.  30.]
 [  1.   1.   1.   0.   0.   0.   9.]
 [ -2.  -1.   0.   1.   0.   0.   4.]
 [  0.   1.   0.   0.  -1.   1.   3.]]
x_B = [3, 4, 2]
[[  4.   0.   0.   0.   5. -15. -15.]
 [  1.   0.   1.   0.   1.  -1.   6.]
 [ -2.   0.   0.   1.  -1.   1.   7.]
 [  0.   1.   0.   0.  -1.   1.   3.]]
x_B = [5, 4, 2]
[[ -1.   0.  -5.   0.   0. -10. -45.]
 [  1.   0.   1.   0.   1.  -1.   6.]
 [ -1.   0.   1.   1.   0.   0.  13.]
 [  1.   1.   1.   0.   0.   0.   9.]]
Delete artificial variables [5]
x_B = [5, 4, 2]
[[ -1.   0.  -5.   0.   0. -45.]
 [  1.   0.   1.   0.   1.   6.]
 [ -1.   0.   1.   1.   0.  13.]
 [  1.   1.   1.   0.   0.   9.]]
Optimal solution: x_opt = [0. 9.]


##### 6. Resolver por el método de dos faces el siguiente LO

In [23]:
A = [[4., 2, -1, 0, 1, 0],
     [1., 4., 0, -1, 0, 1]]

b = [12, 6.0]

c_artificial = [0., 0., 0, 0, -1, -1]
c = [-2, -3., 0, 0]

basic_var = [4, 5]
artificial_var = [4, 5]

tableau = build_tableau(A, b, c_artificial)
tableau = two_phases(tableau, c, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [5, 6]
[[ 0.  0.  0.  0. -1. -1.  0.]
 [ 4.  2. -1.  0.  1.  0. 12.]
 [ 1.  4.  0. -1.  0.  1.  6.]]
Cost corrected
x_B = [5, 6]
[[ 5.  6. -1. -1.  0.  0. 18.]
 [ 4.  2. -1.  0.  1.  0. 12.]
 [ 1.  4.  0. -1.  0.  1.  6.]]
Start phase One
x_B = [5, 2]
[[ 3.5   0.   -1.    0.5   0.   -1.5   9.  ]
 [ 3.5   0.   -1.    0.5   1.   -0.5   9.  ]
 [ 0.25  1.    0.   -0.25  0.    0.25  1.5 ]]
x_B = [1, 2]
[[ 0.          0.          0.          0.         -1.         -1.          0.        ]
 [ 1.          0.         -0.28571429  0.14285714  0.28571429 -0.14285714  2.57142857]
 [ 0.          1.          0.07142857 -0.28571429 -0.07142857  0.28571429  0.85714286]]
Delete artificial variables [4, 5]
Start phase two, put original cost
x_B = [1, 2]
[[-2.         -3.          0.          0.          0.        ]
 [ 1.          0.         -0.28571429  0.14285714  2.57142857]
 [ 0.          1.          0.07142857 -0.28571429  0.85714286]]
Cost corrected
x_B = [1, 2]
[[ 0.          0.         -0.3

In [24]:
A = [[1., 1, 1, 0., 0],
     [-1, 2, 0, -1, 1]]

b = [10, 2.0]

c_artificial = [0., 0., 0, 0, -1]
c = [2, 2., 0, 0]

basic_var = [2, 4]
artificial_var = [4]

tableau = build_tableau(A, b, c_artificial)
tableau = two_phases(tableau, c, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [3, 5]
[[ 0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  0.  0. 10.]
 [-1.  2.  0. -1.  1.  2.]]
Cost corrected
x_B = [3, 5]
[[-1.  2.  0. -1.  0.  2.]
 [ 1.  1.  1.  0.  0. 10.]
 [-1.  2.  0. -1.  1.  2.]]
Start phase One
x_B = [3, 2]
[[ 0.   0.   0.   0.  -1.   0. ]
 [ 1.5  0.   1.   0.5 -0.5  9. ]
 [-0.5  1.   0.  -0.5  0.5  1. ]]
Delete artificial variables [4]
Start phase two, put original cost
x_B = [3, 2]
[[ 2.   2.   0.   0.   0. ]
 [ 1.5  0.   1.   0.5  9. ]
 [-0.5  1.   0.  -0.5  1. ]]
Cost corrected
x_B = [3, 2]
[[ 3.   0.   0.   1.  -2. ]
 [ 1.5  0.   1.   0.5  9. ]
 [-0.5  1.   0.  -0.5  1. ]]
x_B = [1, 2]
[[  0.           0.          -2.           0.         -20.        ]
 [  1.           0.           0.66666667   0.33333333   6.        ]
 [  0.           1.           0.33333333  -0.33333333   4.        ]]
Optimal solution: x_opt = [6. 4.]


##### 7. Aplicar el algoritmo simplex considerando como inicio la solución básica factible asociado al vértice (1, 0) del siguiente LO:

In [25]:
A = [[1, 1, -1, 0, 0, 0, 1],
     [-1, 1, 0, 1, 0, 0, 0],
     [2, 1, 0, 0., 1, 0, 0],
     [1, 0, 0, 0., 0, 1, 0]]

b = [1, 2, 8, 3]

c_artificial = [0, 0, 0, 0, 0, 0, -1.0]
c = [1, 1, 0, 0, 0, 0, 0]

basic_var = [6, 0, 3, 4]
artificial_var = [6]

tableau = build_tableau(A, b, c_artificial)

tableau = two_phases(tableau, c, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [7, 1, 4, 5]
[[ 0.  0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1. -1.  0.  0.  0.  1.  1.]
 [-1.  1.  0.  1.  0.  0.  0.  2.]
 [ 2.  1.  0.  0.  1.  0.  0.  8.]
 [ 1.  0.  0.  0.  0.  1.  0.  3.]]
Cost corrected
x_B = [7, 1, 4, 5]
[[ 3.  1. -1. -1.  1. -1.  0.  4.]
 [ 1.  1. -1.  0.  0.  0.  1.  1.]
 [-1.  1.  0.  1.  0.  0.  0.  2.]
 [ 2.  1.  0.  0.  1.  0.  0.  8.]
 [ 1.  0.  0.  0.  0.  1.  0.  3.]]
Start phase One
x_B = [1, 1, 4, 5]
[[ 0. -2.  2. -1.  1. -1. -3.  1.]
 [ 1.  1. -1.  0.  0.  0.  1.  1.]
 [ 0.  2. -1.  1.  0.  0.  1.  3.]
 [ 0. -1.  2.  0.  1.  0. -2.  6.]
 [ 0. -1.  1.  0.  0.  1. -1.  2.]]
Delete artificial variables [6]
Start phase two, put original cost
x_B = [1, 1, 4, 5]
[[ 1.  1.  0.  0.  0.  0.  0.]
 [ 1.  1. -1.  0.  0.  0.  1.]
 [ 0.  2. -1.  1.  0.  0.  3.]
 [ 0. -1.  2.  0.  1.  0.  6.]
 [ 0. -1.  1.  0.  0.  1.  2.]]
Cost corrected
x_B = [1, 1, 4, 5]
[[ 0.  0.  1.  0.  0.  0. -1.]
 [ 1.  1. -1.  0.  0.  0.  1.]
 [ 0.  2. -1.  1.  0.  0.  3.]
 [ 0. -1.  2.  

9. Resolver por el método de dos faces el siguiente LO

In [26]:
A = [[1, 1, 1, 1., 0, 0.],
     [4, 1, 6, 0, -1, 1]]

b = [1, 2.]

c_artificial = [0., 0., 0, 0, 0, -1]
c = [3, 3., 0, 0, 0.0]

basic_var = [2, 4]
artificial_var = [4]

tableau = build_tableau(A, b, c_artificial)
tableau = two_phases(tableau, c, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [3, 5]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  1.  0.  0.  1.]
 [ 4.  1.  6.  0. -1.  1.  2.]]
Cost corrected
x_B = [3, 5]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  1.  0.  0.  1.]
 [ 4.  1.  6.  0. -1.  1.  2.]]
Start phase One
x_B = [3, 1]
[[ 0.    0.    0.    0.    0.   -1.    0.  ]
 [ 0.    0.75 -0.5   1.    0.25 -0.25  0.5 ]
 [ 1.    0.25  1.5   0.   -0.25  0.25  0.5 ]]
Delete artificial variables [4]
Start phase two, put original cost
x_B = [3, 1]
[[ 3.    3.    0.    0.    0.    0.  ]
 [ 0.    0.75 -0.5   1.   -0.25  0.5 ]
 [ 1.    0.25  1.5   0.    0.25  0.5 ]]
Cost corrected
x_B = [3, 1]
[[ 0.    2.25 -4.5   0.   -0.75 -1.5 ]
 [ 0.    0.75 -0.5   1.   -0.25  0.5 ]
 [ 1.    0.25  1.5   0.    0.25  0.5 ]]
x_B = [2, 1]
[[ 0.          0.         -3.         -3.          0.         -3.        ]
 [ 0.          1.         -0.66666667  1.33333333 -0.33333333  0.66666667]
 [ 1.          0.          1.66666667 -0.33333333  0.33333333  0.33333333]]
Optimal solution: x_opt

### CM4E1 Curso: Práctica Calificada 3

##### 3. Determinar el valor óptimo (si existe solución) del siguiente LO

\begin{align*}
-\max \quad  -x_1& - 2x_2& +3x_3& - 3x_4& \\
\text { s.t. } \quad  3x_1& +2x_2& - x_3& + x_4& + x_5& {}   {}& {} {}& &= 1\\
                      x_1& - 3x_2& + 2x_3& -2x_4& {} {}& - {x_6}& + x_7& &=6 \\
                      x_1&\ge0,&x_2 \ge0,&x_3 \ge0,&x_4 \ge0,& x_5\ge0,&x_6 \ge0,& x_7\ge0&
\end{align*}

In [27]:
A = [[3, 2., -1, 1, 1, 0., 0],
     [1, -3, 2, -2, 0, -1, 1]]
b = [1, 6.]

c = [-1, -2, 3, -3, 0., 0, 0]
c_artificial = [0, 0, 0, 0, 0, 0, -1]

tableau = build_tableau(A, b, c_artificial)

basic_var = [4, 6]
artificial_var = [6]
tableau = two_phases(tableau, c, basic_var, artificial_var)

x_B = [5, 7]
[[ 0.  0.  0.  0.  0.  0. -1.  0.]
 [ 3.  2. -1.  1.  1.  0.  0.  1.]
 [ 1. -3.  2. -2.  0. -1.  1.  6.]]
Cost corrected
x_B = [5, 7]
[[ 1. -3.  2. -2.  0. -1.  0.  6.]
 [ 3.  2. -1.  1.  1.  0.  0.  1.]
 [ 1. -3.  2. -2.  0. -1.  1.  6.]]
Start phase One
x_B = [5, 3]
[[ 0.   0.   0.   0.   0.   0.  -1.   0. ]
 [ 3.5  0.5  0.   0.   1.  -0.5  0.5  4. ]
 [ 0.5 -1.5  1.  -1.   0.  -0.5  0.5  3. ]]
Delete artificial variables [6]
Start phase two, put original cost
x_B = [5, 3]
[[-1.  -2.   3.  -3.   0.   0.   0. ]
 [ 3.5  0.5  0.   0.   1.  -0.5  4. ]
 [ 0.5 -1.5  1.  -1.   0.  -0.5  3. ]]
Cost corrected
x_B = [5, 3]
[[-2.5  2.5  0.   0.   0.   1.5 -9. ]
 [ 3.5  0.5  0.   0.   1.  -0.5  4. ]
 [ 0.5 -1.5  1.  -1.   0.  -0.5  3. ]]
x_B = [2, 3]
[[-20.   0.   0.   0.  -5.   4. -29.]
 [  7.   1.   0.   0.   2.  -1.   8.]
 [ 11.   0.   1.  -1.   3.  -2.  15.]]
The model is unbounded
L = [ 0.  8. 15.  0.  0.  0.] + λ[-0.  1.  2. -0. -0. -0.]
L = [ 0.  8. 15.  0.] + λ[-0.  1.  2. -0

##### 4.	Elaborar un algoritmo que al ingresar una tabla simplex (cuya información es representada en una matriz y un vector que indica cuales son las variables básicas asociadas), indique si es correcto la tabla y obtenga el LO original asociado.

Pseudocódigo
```
Input: tabla_simplex, variables_basicas, holgura=True
1. Verificar que los coeficientes asociados a las variables básicas sean 0.
2. Verificar que Identidad Permutada esté en las variables básicas, i.e. toda la columna es 0 excepto la fila donde está ubicada la variable básica el cual es 1.
3. Verificar que los elementos de la última columna sean mayor o igual a 0 (B^{-1}b >= 0).
```

In [28]:
def check_tableau(tableau: sp.Matrix, basic_var: list[int], holgura = False):
    for index, col in enumerate(basic_var):
        # Verificando que los coeficientes asociados a las variables básicas sean 0.
        if not tableau[0, col] == 0.0:
            print(f"El costo asociado a la variable básica x{col + 1} no es cero.")
            return False

        # Verificando que toda la columna es 0 excepto la fila donde está ubicada la variable básica el cual es 1.
        for row in range(1, len(basic_var) + 1):
            if row != index + 1:
                if not tableau[row, col] == 0.0:
                    print(f"La columna {col} en la fila {row} asociada a la variable básica x{col + 1} no es 0!")
                    return False
            else:
                if not tableau[row, col] == 1.0:
                    print(f"La columna {col} en la fila {row} asociada a la variable básica x{col + 1} no es 1!")
                    return False

        # Verificando que los elementos de la columna fila B^{-1}*b >= 0.
        if tableau[index + 1, -1] < 0.0:
            print(f"La fila {index + 1} de B^{-1} b es negativo")
            return False
    # Si pasa todas las verificaciones es una tabla simplex.
    print("Si es una tabla simplex, cuyo LO original asociado es")
    tableau2standard(tableau, holgura)
    return True


In [29]:
def tableau2standard(tableau: sp.Matrix, holgura = False):
    dim_1, _ = tableau.shape
    # número de restricciones
    m = dim_1 - 1 if not holgura else 0
    simbol = "\le" if not holgura else "="
    # extrayendo las matrices A, b, y c
    c = tableau[0, :-m - 1]
    A = tableau[1:, :-m - 1]
    b = tableau[1:, -1]
    # variables
    x = sp.Matrix([f"x_{i}" for i in range(1, 7 - m)])
    display_latex(Latex(f"""$\max{sp.latex(c.dot(x))}\\\\\\text{{subject to}}\\\\{sp.latex(A)}{sp.latex(x)}{simbol}{sp.latex(b)}$"""))


In [30]:
tableau = sp.Matrix([[3, 2, 0, 0, 0, 0, 0],
                     [1, 1, 1, 0, 0, 0, 9],
                     [3, 1, 0, 1, 0, 0, 18],
                     [1, 0, 0, 0, 1, 0, 7],
                     [0, 1, 0, 0, 0, 1, 6]])

basic_var = [2, 3, 4, 5]
check_tableau(tableau, basic_var)


Si es una tabla simplex, cuyo LO original asociado es


True

In [31]:
tableau = sp.Matrix([[0, 0, -3/2, -1/2, 0, 0, -45/2],
                     [0, 1,  3/2,  1/2, 0, 0, 9/2],
                     [1, 0, -1/2,  1/2, 0, 0, 9/2],
                     [0, 0,  1/2, -1/2, 1, 0, 5/2],
                     [0, 0, -3/2,  1/2, 0, 1, 3/2]])

basic_var = [1, 0, 4, 5]
check_tableau(tableau, basic_var, True)


Si es una tabla simplex, cuyo LO original asociado es


True